## Consolida todas as LDs em um único Dataframe:

In [13]:
# Importações necessárias
import pandas as pd
import os
from pathlib import Path
import numpy as np
import warnings

# Configuração para suprimir avisos
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

In [14]:
import pandas as pd
import os
from pathlib import Path
import numpy as np

def clean_filename(filename):
    # Remove a extensão .xlsx
    filename = filename.replace('.xlsx', '').replace('.XLSX', '')
    # Pega apenas a parte do código do documento (antes do underscore)
    if '_' in filename:
        filename = filename.split('_')[0]
    return filename

# Caminho base para a pasta contendo os arquivos Excel
base_path = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\TodasLDs\\'

# Dataframe para armazenar todos os resultados
df_LD_final = pd.DataFrame()

# Contadores para estatísticas
total_documentos = 0
documentos_com_comentarios = 0
documentos_com_Discipline = 0
cont_files = 0
arquivos_com_erro = []

# Iterar sobre todos os arquivos Excel na pasta
for filename in os.listdir(base_path):
    if not (filename.endswith('.xlsx') or filename.endswith('.XLSX')):
        continue
        
    file_path = os.path.join(base_path, filename)
    cont_files += 1
    print(f"Lendo arquivo: {file_path}")
    
    try:
        # Ler o arquivo Excel
        df = pd.read_excel(file_path, sheet_name="VDRL")
        
        # Pré-processamento dos dados
        df.columns = df.iloc[6]
        df = df.iloc[7:]
        
        # Tratar colunas duplicadas se existirem
        if df.columns.duplicated().any():
            # Criar nomes de colunas únicos
            cols = pd.Series(df.columns)
            for i, col in enumerate(cols):
                if cols.duplicated()[i]:
                    cols.iloc[i] = f"{col}_{cols[:i].tolist().count(col)}"
            df.columns = cols
        
        # Renomear colunas padrão
        column_mapping = {
            "CLIENT DOCUMENT NUMBER": "CLIENT_DOCUMENT",
            "DOCUMENT TITLE": "DOCUMENT_TITLE",
            "Discipline": "Discipline",
            "Comments": "Comments"
        }
        
        # Renomear apenas as colunas que existem
        for old_col, new_col in column_mapping.items():
            if old_col in df.columns:
                df.rename(columns={old_col: new_col}, inplace=True)
        
        # Garantir que as colunas necessárias existem
        for col in ["CLIENT_DOCUMENT", "DOCUMENT_TITLE", "Discipline", "Comments"]:
            if col not in df.columns:
                df[col] = None
        
        # Manter apenas as colunas necessárias
        colunas_para_manter = ["CLIENT_DOCUMENT", "DOCUMENT_TITLE", "Discipline", "Comments"]
        df = df[colunas_para_manter]
        df.reset_index(drop=True, inplace=True)
        
        # Adicionar coluna com o nome completo do arquivo de origem
        df['Arquivo_Original'] = filename
        
        # Adicionar coluna com o nome do arquivo limpo
        df['LD'] = clean_filename(filename)
        
        # Atualizar estatísticas
        total_documentos += len(df)
        documentos_com_comentarios += df['Comments'].notna().sum()
        documentos_com_Discipline += df['Discipline'].notna().sum()
        
        # Concatenar com o DataFrame final
        df_LD_final = pd.concat([df_LD_final, df], axis=0, ignore_index=True)
        
    except Exception as e:
        print(f"Erro ao processar o arquivo {filename}: {str(e)}")
        arquivos_com_erro.append(filename)
        continue

# Resetar o índice do DataFrame final
df_LD_final.reset_index(drop=True, inplace=True)

# Função para limpar e converter para maiúsculas
def clean_text(x):
    if isinstance(x, str) and pd.notna(x):
        return x.strip().upper()
    return x

# Aplicar a função nas colunas
df_LD_final = df_LD_final.dropna(subset=['DOCUMENT_TITLE'])
df_LD_final['DOCUMENT_TITLE'] = df_LD_final['DOCUMENT_TITLE'].apply(clean_text)
df_LD_final['Discipline'] = df_LD_final['Discipline'].apply(clean_text)
df_LD_final['Comments'] = df_LD_final['Comments'].apply(clean_text)

# Função para substituir valores com "YES"
def convert_yes(value):
    if isinstance(value, str):
        value_upper = value.upper().strip()
        if 'YES' in value_upper or value_upper == 'Y' or value_upper == 'X' or value_upper == 'SIM':
            return 'YES'
    return ''

# Criar nova coluna 'Comments_new'
df_LD_final['Comments_new'] = df_LD_final['Comments'].apply(convert_yes)

# Atualizar estatísticas
total_documentos = len(df_LD_final)
documentos_com_comentarios = (df_LD_final['Comments_new'] == 'YES').sum()
documentos_com_Discipline = df_LD_final['Discipline'].notna().sum()

# Calcular estatísticas finais
if total_documentos > 0:
    porcentagem_com_comentarios = (documentos_com_comentarios / total_documentos) * 100
    porcentagem_com_Discipline = (documentos_com_Discipline / total_documentos) * 100
else:
    porcentagem_com_comentarios = 0
    porcentagem_com_Discipline = 0

# Criar DataFrame de estatísticas
df_estatisticas = pd.DataFrame({
    'Métrica': ['Total de Documentos', 'Documentos com YES', 'Documentos com Discipline',
                '% com Comentários', '% com Discipline', 'Arquivos com erro'],
    'Valor': [total_documentos, documentos_com_comentarios, documentos_com_Discipline,
              f"{porcentagem_com_comentarios:.2f}%", f"{porcentagem_com_Discipline:.2f}%", 
              len(arquivos_com_erro)]
})

# Salvar o DataFrame final e as estatísticas
output_file = 'C:\\Users\\elxy\\Documents\\Codigos\\Python\\P84_85\\LDs\\LD_comment_combined.xlsx'

with pd.ExcelWriter(output_file) as writer:
    df_LD_final.to_excel(writer, sheet_name='LD', index=False)
    df_estatisticas.to_excel(writer, sheet_name='Estatísticas', index=False)
    
    # Adicionar lista de arquivos com erro
    if arquivos_com_erro:
        df_erros = pd.DataFrame({'Arquivos com erro': arquivos_com_erro})
        df_erros.to_excel(writer, sheet_name='Erros', index=False)

print(f'Arquivo {output_file} salvo em: {Path(output_file).resolve()}')

# Imprimir estatísticas no console
print("\nMétricas:")
print(f"Número de arquivos processados: {cont_files}")
print(f"Total de Documentos: {total_documentos}")
print(f"Documentos com YES: {documentos_com_comentarios} ({porcentagem_com_comentarios:.2f}%)")
print(f"Documentos com Discipline: {documentos_com_Discipline} ({porcentagem_com_Discipline:.2f}%)")
print(f"Arquivos com erro: {len(arquivos_com_erro)}")
if arquivos_com_erro:
    print("Lista de arquivos com erro:")
    for arquivo in arquivos_com_erro:
        print(f"  - {arquivo}")


Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-210-YA7-001_RISER HARD PIPE.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-830-CT6-001_0_CHOKE VALVES.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-EP8-096_0_FLOW METERING SYSTEM - rev1.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-FK1-001_0_AXIAL_SDV.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-HG8-001_PRESSURE VACUUM RELIEF VALVE.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-HM9-001_0_PRESSURE VESSELS_TS.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-911-HYE-001_NON_METALLIC_TANKS.xlsx
Lendo arquivo: C:\Users\elxy\Documents\Codigos\Python\P84_85\LDs\TodasLDs\I-LD-3010.2S-1200-91

In [15]:
# Define os caminhos dos arquivos
bd_path = 'C:\\Users\\ELXY\\Documents\\Codigos\\Python\\P84_85\\DadosBasicos\\'
orbis_path = 'C:\\Users\\ELXY\\Documents\\Codigos\\Python\\P84_85\\'

# Define o nome dos arquivos
bd_file = 'dados_basicos.xlsx'
orbis_file = 'orbis.xlsx'

df_db = pd.read_excel(os.path.join(os.path.dirname(bd_path), bd_file), sheet_name="LDxPKG")
df_orbis = pd.read_excel(os.path.join(os.path.dirname(orbis_path), orbis_file), sheet_name="Export")

# listar as linhas que tem PACKAGE duplicado
# df_db_duplicates = df_db[df_db.duplicated(subset=['PACKAGE'], keep=False)]
# Exibir as linhas duplicadas
#print("Linhas com PACKAGE duplicado:")
# display(df_db_duplicates)

# criar uma nova coluna chamada 'PKG_OK' no no df_db para verificar se o nome da coluna 'PACKAGE' existe no df_orbis na 'PKG DESCRIPTION'
df_db['PKG_OK'] = df_db['PACKAGE'].isin(df_orbis['PKG DESCRIPTION'])

#with pd.ExcelWriter("db.xlsx") as writer:
#    df_db.to_excel(writer, sheet_name='dados', index=False)

# Inserir as colunas 'Origem' e 'PACKAGE' no DataFrames df_LD_final mapeando as colunas LD do dataframe df_db com a coluna LD do dataframe df_LD_infal
df_LD_final['Origem'] = df_LD_final['LD'].map(df_db.set_index('LD')['Origem'])
df_LD_final['PACKAGE'] = df_LD_final['LD'].map(df_db.set_index('LD')['PACKAGE'])

In [16]:
# Remover do dataframe df_LD_final a coluna "Comments" não necessária
df_LD_final = df_LD_final.drop(columns=['Comments'])

# Ordenar as colunas do dataframe df_LD_final por CLIENT_DOCUMENT	DOCUMENT_TITLE	LD	Discipline Comments_new	PACKAGE Origem
df_LD_final = df_LD_final[['CLIENT_DOCUMENT', 'DOCUMENT_TITLE', 'LD', 'Discipline', 'Comments_new', 'PACKAGE', 'Origem']]

# Exibir o DataFrame final após remoção da coluna "Comments" e reordenação das colunas
df_LD_final.info()

# Salvar resultados
with pd.ExcelWriter(output_file) as writer:
    df_LD_final.to_excel(writer, sheet_name='LD', index=False)

<class 'pandas.core.frame.DataFrame'>
Index: 16501 entries, 0 to 16843
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   CLIENT_DOCUMENT  16360 non-null  object
 1   DOCUMENT_TITLE   16501 non-null  object
 2   LD               16501 non-null  object
 3   Discipline       8793 non-null   object
 4   Comments_new     16501 non-null  object
 5   PACKAGE          15849 non-null  object
 6   Origem           15849 non-null  object
dtypes: object(7)
memory usage: 1.0+ MB


### Avalia dos documentos emitidos com VDA quais foram solicitados para comentar

#### Importa bibliotecas e define nome dos arquivos

In [1]:
import pandas as pd
import os
from datetime import datetime

# Define os caminhos dos arquivos
vda_path = 'C:\\Users\\ELXY\\Documents\\Codigos\\Python\\P84_85\\LDs\\TodasVDAs\\'
ld_path = 'C:\\Users\\ELXY\\Documents\\Codigos\\Python\\P84_85\\LDs\\'

# Get today's date and format it as YY.MM.DD
today = datetime.now()
vda_file = f"VDA {today.strftime('%y.%m.%d')}.xlsx"
ld_file = 'LD_comment_combined.xlsx'
vda_updated_file = f"VDA {today.strftime('%y.%m.%d')}_updated.xlsx"
vda_updated_file_op = f"VDA {today.strftime('%y.%m.%d')}_updated_op.xlsx"


#### Analisa se o documento existe na LD consolidada.
- Verifica se cada documento referênciado nas VDAs tem sua correspondência na LD Consolidada.
- Verifica se existem comentários (YES/NO) nesses documentos.
- Verifica também qual a Discipline do documento.

In [2]:
# Carregar os arquivos Excel
vda_df = pd.read_excel(os.path.join(os.path.dirname(vda_path), vda_file), sheet_name="DADOS")
ld_df = pd.read_excel(os.path.join(os.path.dirname(ld_path), ld_file), sheet_name="LD")

# Criar as novas colunas no DataFrame VDA
vda_df['Discipline'] = ''
vda_df['Comments_new'] = ''

# Iterar sobre as linhas do DataFrame VDA
for index, row in vda_df.iterrows():
    ref_doc = row['REFERENCE DOCUMENT NAME']
    
    # Procurar o documento correspondente no DataFrame LD
    matching_row = ld_df[ld_df['CLIENT_DOCUMENT'] == ref_doc]
    
    if not matching_row.empty:
        # Se encontrou uma correspondência
        vda_df.at[index, 'Discipline'] = matching_row['Discipline'].values[0]
        
        if pd.notna(matching_row['Comments_new'].values[0]):
            vda_df.at[index, 'Comments_new'] = matching_row['Comments_new'].values[0]
        else:
            vda_df.at[index, 'Comments_new'] = 'NO'
    else:
        # Se não encontrou correspondência
        vda_df.at[index, 'Comments_new'] = 'Documento não encontrado na LD_comment_combined.xlsx'

# Remover linhas duplicadas com base na coluna "VDA NAME"
if "VDA NAME" in vda_df.columns and "VDA VERSION" in vda_df.columns:
    print(f"Número de linhas antes da remoção de duplicatas: {len(vda_df)}")
    # Sort by VDA VERSION in descending order and keep first occurrence of each VDA NAME
    vda_df = vda_df.sort_values('VDA VERSION', ascending=False).drop_duplicates(subset='VDA NAME', keep='first')
    print(f"Número de linhas após a remoção de duplicatas: {len(vda_df)}")
else:
    print("Aviso: Colunas necessárias não encontradas. Não foi possível remover duplicatas.")


Número de linhas antes da remoção de duplicatas: 613
Número de linhas após a remoção de duplicatas: 612


In [3]:
# Cria uma máscara contendo 'ELECTRICAL' ou 'ELETRICA' na 'Discipline'
electrical_mask = vda_df['Discipline'].str.contains('ELECTRICAL|ELETRICA|ELÉTRICA', case=False, na=False)
# Cria um novo dataframe com as linhas filtradas
df_electrical = vda_df[electrical_mask]
# Manter somente as linhas do dataframe df_electrical onde a coluna '>8WORKING DAYS' é igual a 'x'
df_electrical = df_electrical[df_electrical['>8WORKING DAYS'].str.lower() == 'x']
# Reset index
df_electrical.reset_index(drop=True, inplace=True)

# Cria uma máscara contendo AUTOMATION|INSTRUMENTATION|INSTRUMENTAÇÃO na 'Discipline'
automation_mask = vda_df['Discipline'].str.contains('AUTOMATION|AUTOMAÇÃO|A&I|INSTRUMENTATION|INSTRUMENTAÇÃO|AUT&INST', case=False, na=False)
# Cria um novo dataframe com as linhas filtradas
df_automation = vda_df[automation_mask]
# Manter somente as linhas do dataframe df_automation onde a coluna '>8WORKING DAYS' é igual a 'x'
df_automation = df_automation[df_automation['>8WORKING DAYS'].str.lower() == 'x']
# Reset index
df_automation.reset_index(drop=True, inplace=True)

# Cria uma máscara contendo 'STATIC|ESTATICO|ESTÁTICO' na 'Discipline'
static_mask = vda_df['Discipline'].str.contains('STATIC|ESTATICO|ESTÁTICO', case=False, na=False)
# Cria um novo dataframe com as linhas filtradas
df_static = vda_df[static_mask]
# Manter somente as linhas do dataframe df_static onde a coluna '>8WORKING DAYS' é igual a 'x'
df_static = df_static[df_static['>8WORKING DAYS'].str.lower() == 'x']
# Reset index
df_static.reset_index(drop=True, inplace=True)

# Cria uma máscara contendo 'PIPING|TUBULAÇÃO' na 'Discipline'
piping_mask = vda_df['Discipline'].str.contains('PIPING|TUBULAÇÃO', case=False, na=False)
# Cria um novo dataframe com as linhas filtradas
df_piping = vda_df[piping_mask]
# Manter somente as linhas do dataframe df_piping onde a coluna '>8WORKING DAYS' é igual a 'x'
df_piping = df_piping[df_piping['>8WORKING DAYS'].str.lower() == 'x']
# Reset index
df_piping.reset_index(drop=True, inplace=True)

# Cria uma máscara contendo 'TELECOM|TELECOMUNICAÇÃO|TELECOMUNICAÇÕES' na 'Discipline'
telecom_mask = vda_df['Discipline'].str.contains('TELECOM|TELECOMUNICAÇÃO|TELECOMUNICAÇÕES', case=False, na=False)
# Cria um novo dataframe com as linhas filtradas
df_telecom = vda_df[telecom_mask]
# Manter somente as linhas do dataframe df_telecom onde a coluna '>8WORKING DAYS' é igual a 'x'
df_telecom = df_telecom[df_telecom['>8WORKING DAYS'].str.lower() == 'x']
# Reset index
df_telecom.reset_index(drop=True, inplace=True)

# Cria uma máscara contendo 'OPERATION|UN|OPERAÇÃO' na 'Discipline'
operation_mask = vda_df['Discipline'].str.contains('OPERATION|UN|OPERAÇÃO', case=False, na=False)
# Cria um novo dataframe com as linhas filtradas
df_operation = vda_df[operation_mask]
# Manter somente as linhas do dataframe df_operation onde a coluna '>8WORKING DAYS' é igual a 'x'
df_operation = df_operation[df_operation['>8WORKING DAYS'].str.lower() == 'x']
# Reset index
df_operation.reset_index(drop=True, inplace=True)

# Cria uma máscara contendo 'COMISSIONING|EICOM|COMM|COMISSIONAMENTO' na 'Discipline'
commissioning_mask = vda_df['Discipline'].str.contains('COMISSIONING|EICOM|COMM|COMISSIONAMENTO', case=False, na=False)
# Cria um novo dataframe com as linhas filtradas
df_commissioning = vda_df[commissioning_mask]
# Manter somente as linhas do dataframe df_commissioning onde a coluna '>8WORKING DAYS' é igual a 'x'
df_commissioning = df_commissioning[df_commissioning['>8WORKING DAYS'].str.lower() == 'x']
# Reset index
df_commissioning.reset_index(drop=True, inplace=True)

# Salvar o DataFrame VDA atualizado
output_path = os.path.join(os.path.dirname(vda_path), vda_updated_file)

with pd.ExcelWriter(output_path) as writer:
    vda_df.to_excel(writer, sheet_name='VDA', index=False)
    df_electrical.to_excel(writer, sheet_name='Eletrica', index=False)
    df_automation.to_excel(writer, sheet_name='Automation', index=False)
    df_static.to_excel(writer, sheet_name='Static', index=False)
    df_piping.to_excel(writer, sheet_name='Piping', index=False)
    df_telecom.to_excel(writer, sheet_name='Telecom', index=False)
    df_operation.to_excel(writer, sheet_name='Operation', index=False)
    df_commissioning.to_excel(writer, sheet_name='Commissioning', index=False)

print(f"Arquivo atualizado salvo em: {output_path}")



Arquivo atualizado salvo em: C:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs\TodasVDAs\VDA 25.06.30_updated.xlsx


In [4]:
# Cria uma máscara onde a coluna 'Discipline' é exatamente igual a "OPERATION", "UN" ou "OPERAÇÃO"
operation_mask_2 = vda_df['Discipline'].str.strip().str.upper().isin(['OPERATION', 'UN', 'OPERAÇÃO'])
# Cria um novo dataframe com as linhas filtradas
df_operation_2 = vda_df[operation_mask_2]
# Manter somente as linhas do dataframe df_operation_2 onde a coluna '>8WORKING DAYS' é igual a 'x'
df_operation_2 = df_operation_2[df_operation_2['>8WORKING DAYS'].str.lower() == 'x']
# Reset index
df_operation_2.reset_index(drop=True, inplace=True)

# Cria uma máscara contendo 'COMISSIONING|EICOM|COMM|COMISSIONAMENTO' na 'Discipline'
not_find_mask = vda_df['Comments_new'].str.contains('NO|Documento não encontrado na LD_comment_combined.xlsx', case=False, na=False)
# Cria um novo dataframe com as linhas filtradas
df_not_find = vda_df[not_find_mask]
# Manter somente as linhas do dataframe df_commissioning onde a coluna '>8WORKING DAYS' é igual a 'x'
df_not_find = df_not_find[df_not_find['>8WORKING DAYS'].str.lower() == 'x']
# Reset index
df_not_find.reset_index(drop=True, inplace=True)

# Salvar o DataFrame VDA atualizado
output_path = os.path.join(os.path.dirname(vda_path), vda_updated_file_op)

with pd.ExcelWriter(output_path) as writer:
    df_operation_2.to_excel(writer, sheet_name='Operation', index=False)
    df_not_find.to_excel(writer, sheet_name='No_Not_Find', index=False)

print(f"Arquivo atualizado salvo em: {output_path}")


Arquivo atualizado salvo em: C:\Users\ELXY\Documents\Codigos\Python\P84_85\LDs\TodasVDAs\VDA 25.06.30_updated_op.xlsx
